In [1]:
def divide_map_bounds(west, east, south, north, num_lon_splits, num_lat_splits):
    lon_step = (east - west) / num_lon_splits
    lat_step = (north - south) / num_lat_splits
    grids = []
    for i in range(num_lon_splits):
        for j in range(num_lat_splits):
            grid_west = west + (i * lon_step)
            grid_east = grid_west + lon_step
            grid_south = south + (j * lat_step)
            grid_north = grid_south + lat_step

            grids.append({
                'west': grid_west,
                'east': grid_east,
                'south': grid_south,
                'north': grid_north
            })

    return grids

mapBounds = {
    'west': -80.28036812148592,
    'east': -79.89824989638826,
    'south': 35.75662050463715,
    'north': 35.95251905496488,
}
splitted_coords = divide_map_bounds(**mapBounds, num_lon_splits=6, num_lat_splits=5)
splitted_coords

[{'west': -80.28036812148592,
  'east': -80.21668175063631,
  'south': 35.75662050463715,
  'north': 35.795800214702695},
 {'west': -80.28036812148592,
  'east': -80.21668175063631,
  'south': 35.795800214702695,
  'north': 35.83497992476824},
 {'west': -80.28036812148592,
  'east': -80.21668175063631,
  'south': 35.83497992476824,
  'north': 35.87415963483379},
 {'west': -80.28036812148592,
  'east': -80.21668175063631,
  'south': 35.87415963483379,
  'north': 35.913339344899335},
 {'west': -80.28036812148592,
  'east': -80.21668175063631,
  'south': 35.913339344899335,
  'north': 35.95251905496488},
 {'west': -80.21668175063631,
  'east': -80.1529953797867,
  'south': 35.75662050463715,
  'north': 35.795800214702695},
 {'west': -80.21668175063631,
  'east': -80.1529953797867,
  'south': 35.795800214702695,
  'north': 35.83497992476824},
 {'west': -80.21668175063631,
  'east': -80.1529953797867,
  'south': 35.83497992476824,
  'north': 35.87415963483379},
 {'west': -80.21668175063631,

In [2]:
responses = []
json_payloads = []
for i in splitted_coords:
    j_data = {
        'searchQueryState': {
            'pagination': {
                'currentPage': 21,
            },
            'isMapVisible': True,
            'mapBounds':i,
            'regionSelection': [
                {
                    'regionId': 36,
                    'regionType': 2,
                },
            ],
            'filterState': {
                'sortSelection': {
                    'value': 'globalrelevanceex',
                },
                'isForSaleByAgent': {
                    'value': False,
                },
                'isForSaleByOwner': {
                    'value': False,
                },
                'isNewConstruction': {
                    'value': False,
                },
                'isComingSoon': {
                    'value': False,
                },
                'isAuction': {
                    'value': False,
                },
                'isForSaleForeclosure': {
                    'value': False,
                },
                'isRecentlySold': {
                    'value': True,
                },
            },
            'isListVisible': True,
            'mapZoom': 6,
        },
        'wants': {
            'cat1': [
                'listResults',
                'mapResults',
            ],
        },
        'requestId': 11,
        'isDebugRequest': False,
    }
    json_payloads.append(j_data)
len(json_payloads)

30

In [3]:
import re
import requests
from curl_cffi import requests as curl_r
import json
import pandas as pd
import time, random

cookies = {
    '_hp2_id.1215457233': '%7B%22userId%22%3A%222230556399004484%22%2C%22pageviewId%22%3A%227996225023036108%22%2C%22sessionId%22%3A%222432367684716351%22%2C%22identity%22%3Anull%2C%22trackerVersion%22%3A%224.0%22%7D',
    'zguid': '24|%24bfec1c83-95a4-4ebd-a83f-ed8c3e32a6f0',
    'zjs_anonymous_id': '%22bfec1c83-95a4-4ebd-a83f-ed8c3e32a6f0%22',
    'zjs_user_id': 'null',
    'zg_anonymous_id': '%224e64cb82-f529-43d3-b1d0-bc0fbc6e91f9%22',
    '_pxvid': '37af2b21-cf4e-11ef-8b7c-f8ee72dc674e',
    'zgsession': '1|c6136cdb-2a3d-43c4-a13e-30f674889b3f',
    'pxcts': '7ebf61ea-cf7d-11ef-bd49-814e23b88f59',
    'JSESSIONID': '6701FC2E2A7103B157F9A77E282C8713',
    'AWSALB': '6mRIvoZkCIKT6Z7SBTr64wDVFtIpevFu+h7zIUGtuGS+OzK6aWYHmlP+o1dMAicFA/CkHxanHT1Z5LZfJ2dUyaTbaKL52rAij2CnxmoBAlW8HwjwQvvp60SxIDcb',
    'AWSALBCORS': '6mRIvoZkCIKT6Z7SBTr64wDVFtIpevFu+h7zIUGtuGS+OzK6aWYHmlP+o1dMAicFA/CkHxanHT1Z5LZfJ2dUyaTbaKL52rAij2CnxmoBAlW8HwjwQvvp60SxIDcb',
    '_px3': 'cac93eabba8935c2ec8af0a9a2a09406e6106b40c1683a92437ce87cabe5b5ef:UpNf/MXfpWi4WU8GIezSkqrZywDgcaVvTHcElEMsjWeZ/3npZvR8GEtC7V9g21+9MpziFD6VvxW+qlwCXQbvUg==:1000:3iWkVXZfmCXyphRkM9dPBriCPx4t3lk5PyDID6T4n+PhhQb2WnKlZ8XSsXtyRNdnPmXQwkKZlevJJDWbOSbrdoH3JNuWyqoAbTrOqt/7W0sjOQNSyfd6eEWfFNbJOYB1XHI4F4rH4oftUB7jZd+V+3gF9cY39Ym0EXICoGVvxrobx4KXmIiQeRm6n0C7TchP+OcHQfKXl07z22Om7PIFWhaCoRVo5grazf4EmwCPp2c=',
    'search': '6|1739125908779%7Crect%3D38.71989219561046%2C-67.633210796875%2C31.48498897341223%2C-92.088777203125%26rid%3D36%26disp%3Dmap%26mdm%3Dauto%26p%3D7%26listPriceActive%3D1%26fs%3D0%26fr%3D0%26mmm%3D0%26rs%3D1%26singlestory%3D0%26housing-connector%3D0%26parking-spots%3Dnull-%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%26showcase%3D0%26featuredMultiFamilyBuilding%3D0%26onlyRentalStudentHousingType%3D0%26onlyRentalIncomeRestrictedHousingType%3D0%26onlyRentalMilitaryHousingType%3D0%26onlyRentalDisabledHousingType%3D0%26onlyRentalSeniorHousingType%3D0%26commuteMode%3Ddriving%26commuteTimeOfDay%3Dnow%09%0936%09%7B%22isList%22%3Atrue%2C%22isMap%22%3Atrue%7D%09%09%09%09%09',
}
headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,ar;q=0.8',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'origin': 'https://www.zillow.com',
    'pragma': 'no-cache',
    'priority': 'u=1, i',
    'referer': 'https://www.zillow.com/nc/sold/7_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A7%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-92.088777203125%2C%22east%22%3A-67.633210796875%2C%22south%22%3A32.768894735707235%2C%22north%22%3A37.52724251059601%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A36%2C%22regionType%22%3A2%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A6%7D',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
}
json_data = {
    'searchQueryState': {
        'pagination': {
            'currentPage': 21,
        },
        'isMapVisible': True,
        'mapBounds': {
            'west': -92.088777203125,
            'east': -79.860994,
            'south': 32.768894735707235,
            'north': 35.14806862315162
        },
        'regionSelection': [
            {
                'regionId': 36,
                'regionType': 2,
            },
        ],
        'filterState': {
            'sortSelection': {
                'value': 'globalrelevanceex',
            },
            'isForSaleByAgent': {
                'value': False,
            },
            'isForSaleByOwner': {
                'value': False,
            },
            'isNewConstruction': {
                'value': False,
            },
            'isComingSoon': {
                'value': False,
            },
            'isAuction': {
                'value': False,
            },
            'isForSaleForeclosure': {
                'value': False,
            },
            'isRecentlySold': {
                'value': True,
            },
        },
        'isListVisible': True,
        'mapZoom': 6,
    },
    'wants': {
        'cat1': [
            'listResults',
            'mapResults',
        ],
    },
    'requestId': 11,
    'isDebugRequest': False,
}

In [6]:
for j in json_payloads:
    response = curl_r.put('https://www.zillow.com/async-create-search-page-state', cookies=cookies, headers=headers, json=j, impersonate='chrome124')
    try: 
        print("Found items:", len(response.json()['cat1']['searchResults']['mapResults']))
        responses.extend(response.json()['cat1']['searchResults']['mapResults'])
        time.sleep(random.randint(2, 6))
    except KeyError:
        print("Non-existing key. Available keys are:\n", response.json().keys())    
print(len(responses))

Found items: 205
Found items: 500
Found items: 500
Found items: 351
Found items: 128
Found items: 117
Found items: 132
Found items: 88
Found items: 45
Found items: 94
Found items: 121
Found items: 133
Found items: 300
Found items: 283
Found items: 132
Found items: 75
Found items: 170
Found items: 500
Found items: 500
Found items: 500
Found items: 92
Found items: 91
Found items: 146
Found items: 500


SSLError: Failed to perform, curl: (35) Recv failure: Connection was reset. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.

In [8]:
df = pd.DataFrame(responses)
df

,zpid,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,...,minArea,availabilityDate,marketingTreatments,lotId,badgeInfo,plid,communityName,isCdpResult,isComingSoonCommunity,isR4R
0,442326242,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/f021f027a52...,True,/homedetails/744-Brown-St-3-Lexington-NC-27292...,SOLD,Sold,"$239,990",$240K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,416226596,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/8ee222b2cfb...,True,/homedetails/865-Old-Linwood-Rd-Lexington-NC-2...,SOLD,Sold,"$287,697",$288K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,53137528,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/7d5d2186ea8...,True,/homedetails/310-Kimberly-Ln-Lexington-NC-2729...,SOLD,Sold,"$255,000",$255K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,442326241,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/2a2f4150d75...,True,/homedetails/714-Brown-St-6-Lexington-NC-27292...,SOLD,Sold,"$264,990",$265K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,442326240,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/c107cf0e131...,True,/homedetails/674-Brown-St-10-Lexington-NC-2729...,SOLD,Sold,"$264,990",$265K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5698,50083104,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/7279c780790...,True,/homedetails/133-Barwood-Ter-Trinity-NC-27370/...,SOLD,Sold,"$205,000",$205K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5699,206392029,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/78041b38496...,True,/homedetails/1206-Westbrook-Ct-Archdale-NC-272...,SOLD,Sold,"$128,500",$129K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5700,50082041,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/6d3748fe136...,True,/homedetails/4209-Barrett-Dr-Archdale-NC-27263...,SOLD,Sold,"$169,500",$170K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5701,50085349,RecentlySold,Non Owner Occupied,https://maps.googleapis.com/maps/api/staticmap...,NaN,/homedetails/5045-Brookdale-Dr-Trinity-NC-2737...,SOLD,Sold,"$66,000",$66K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
to_keep = ['address', 'price', 'beds', 'baths', 'area', 'detailUrl']
df = df[to_keep]
df

,address,price,beds,baths,area,detailUrl
0,"744 Brown St #3, Lexington, NC 27292","$239,990",3.0,3.0,1383.0,/homedetails/744-Brown-St-3-Lexington-NC-27292...
1,"865 Old Linwood Rd, Lexington, NC 27292","$287,697",4.0,3.0,2204.0,/homedetails/865-Old-Linwood-Rd-Lexington-NC-2...
2,"310 Kimberly Ln, Lexington, NC 27292","$255,000",3.0,2.0,1758.0,/homedetails/310-Kimberly-Ln-Lexington-NC-2729...
3,"714 Brown St #6, Lexington, NC 27292","$264,990",4.0,3.0,1857.0,/homedetails/714-Brown-St-6-Lexington-NC-27292...
4,"674 Brown St #10, Lexington, NC 27292","$264,990",4.0,3.0,1857.0,/homedetails/674-Brown-St-10-Lexington-NC-2729...
...,...,...,...,...,...,...
5698,"133 Barwood Ter, Trinity, NC 27370","$205,000",3.0,2.0,1366.0,/homedetails/133-Barwood-Ter-Trinity-NC-27370/...
5699,"1206 Westbrook Ct, Archdale, NC 27263","$128,500",2.0,2.0,842.0,/homedetails/1206-Westbrook-Ct-Archdale-NC-272...
5700,"4209 Barrett Dr, Archdale, NC 27263","$169,500",3.0,2.0,1549.0,/homedetails/4209-Barrett-Dr-Archdale-NC-27263...
5701,"5045 Brookdale Dr, Trinity, NC 27370","$66,000",2.0,1.5,1120.0,/homedetails/5045-Brookdale-Dr-Trinity-NC-2737...


In [10]:
df = df.drop_duplicates()
df.shape

(5703, 6)

In [39]:
df.loc[df['baths'] == 1.5, 'link'].iloc[0]

'https://www.zillow.com//homedetails/1132-Owens-Rd-Lexington-NC-27292/5712979_zpid/'

In [11]:
df.loc[:, 'price'] = df.loc[:, 'price'].apply(lambda row: re.sub(r'[^\d]', '', row))
df

C:\Users\Mido Hany\AppData\Local\Temp\ipykernel_5456\3280207385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'price'] = df.loc[:, 'price'].apply(lambda row: re.sub(r'[^\d]', '', row))


,address,price,beds,baths,area,detailUrl
0,"744 Brown St #3, Lexington, NC 27292",239990,3.0,3.0,1383.0,/homedetails/744-Brown-St-3-Lexington-NC-27292...
1,"865 Old Linwood Rd, Lexington, NC 27292",287697,4.0,3.0,2204.0,/homedetails/865-Old-Linwood-Rd-Lexington-NC-2...
2,"310 Kimberly Ln, Lexington, NC 27292",255000,3.0,2.0,1758.0,/homedetails/310-Kimberly-Ln-Lexington-NC-2729...
3,"714 Brown St #6, Lexington, NC 27292",264990,4.0,3.0,1857.0,/homedetails/714-Brown-St-6-Lexington-NC-27292...
4,"674 Brown St #10, Lexington, NC 27292",264990,4.0,3.0,1857.0,/homedetails/674-Brown-St-10-Lexington-NC-2729...
...,...,...,...,...,...,...
5698,"133 Barwood Ter, Trinity, NC 27370",205000,3.0,2.0,1366.0,/homedetails/133-Barwood-Ter-Trinity-NC-27370/...
5699,"1206 Westbrook Ct, Archdale, NC 27263",128500,2.0,2.0,842.0,/homedetails/1206-Westbrook-Ct-Archdale-NC-272...
5700,"4209 Barrett Dr, Archdale, NC 27263",169500,3.0,2.0,1549.0,/homedetails/4209-Barrett-Dr-Archdale-NC-27263...
5701,"5045 Brookdale Dr, Trinity, NC 27370",66000,2.0,1.5,1120.0,/homedetails/5045-Brookdale-Dr-Trinity-NC-2737...


In [12]:
df['link'] = df['detailUrl'].apply(lambda x: f"https://www.zillow.com/{x}")
df.loc[100, :]

C:\Users\Mido Hany\AppData\Local\Temp\ipykernel_5456\728619470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['link'] = df['detailUrl'].apply(lambda x: f"https://www.zillow.com/{x}")


address                 113 S Avondale Rd, Lexington, NC 27292
price                                                   155000
beds                                                       3.0
baths                                                      1.0
area                                                    1578.0
detailUrl    /homedetails/113-S-Avondale-Rd-Lexington-NC-27...
link         https://www.zillow.com//homedetails/113-S-Avon...
Name: 100, dtype: object

In [13]:
df = df.drop(columns='detailUrl')
df

,address,price,beds,baths,area,link
0,"744 Brown St #3, Lexington, NC 27292",239990,3.0,3.0,1383.0,https://www.zillow.com//homedetails/744-Brown-...
1,"865 Old Linwood Rd, Lexington, NC 27292",287697,4.0,3.0,2204.0,https://www.zillow.com//homedetails/865-Old-Li...
2,"310 Kimberly Ln, Lexington, NC 27292",255000,3.0,2.0,1758.0,https://www.zillow.com//homedetails/310-Kimber...
3,"714 Brown St #6, Lexington, NC 27292",264990,4.0,3.0,1857.0,https://www.zillow.com//homedetails/714-Brown-...
4,"674 Brown St #10, Lexington, NC 27292",264990,4.0,3.0,1857.0,https://www.zillow.com//homedetails/674-Brown-...
...,...,...,...,...,...,...
5698,"133 Barwood Ter, Trinity, NC 27370",205000,3.0,2.0,1366.0,https://www.zillow.com//homedetails/133-Barwoo...
5699,"1206 Westbrook Ct, Archdale, NC 27263",128500,2.0,2.0,842.0,https://www.zillow.com//homedetails/1206-Westb...
5700,"4209 Barrett Dr, Archdale, NC 27263",169500,3.0,2.0,1549.0,https://www.zillow.com//homedetails/4209-Barre...
5701,"5045 Brookdale Dr, Trinity, NC 27370",66000,2.0,1.5,1120.0,https://www.zillow.com//homedetails/5045-Brook...


In [14]:
df['price'] = pd.to_numeric(df['price'], errors='coerce')

df.dtypes

address     object
price      float64
beds       float64
baths      float64
area       float64
link        object
dtype: object

In [15]:
df = df.fillna('-')

In [16]:
df

,address,price,beds,baths,area,link
0,"744 Brown St #3, Lexington, NC 27292",239990.0,3.0,3.0,1383.0,https://www.zillow.com//homedetails/744-Brown-...
1,"865 Old Linwood Rd, Lexington, NC 27292",287697.0,4.0,3.0,2204.0,https://www.zillow.com//homedetails/865-Old-Li...
2,"310 Kimberly Ln, Lexington, NC 27292",255000.0,3.0,2.0,1758.0,https://www.zillow.com//homedetails/310-Kimber...
3,"714 Brown St #6, Lexington, NC 27292",264990.0,4.0,3.0,1857.0,https://www.zillow.com//homedetails/714-Brown-...
4,"674 Brown St #10, Lexington, NC 27292",264990.0,4.0,3.0,1857.0,https://www.zillow.com//homedetails/674-Brown-...
...,...,...,...,...,...,...
5698,"133 Barwood Ter, Trinity, NC 27370",205000.0,3.0,2.0,1366.0,https://www.zillow.com//homedetails/133-Barwoo...
5699,"1206 Westbrook Ct, Archdale, NC 27263",128500.0,2.0,2.0,842.0,https://www.zillow.com//homedetails/1206-Westb...
5700,"4209 Barrett Dr, Archdale, NC 27263",169500.0,3.0,2.0,1549.0,https://www.zillow.com//homedetails/4209-Barre...
5701,"5045 Brookdale Dr, Trinity, NC 27370",66000.0,2.0,1.5,1120.0,https://www.zillow.com//homedetails/5045-Brook...


In [17]:
df.to_csv('final_homes.csv', index=False)